In [36]:
from scipy.io import loadmat
import h5py
import numpy as np

In [37]:
exptSummary = loadmat('/root/capsule/scratch/Matlab/scan_00003_20230815_162233_DENOISED_EXPTSUMMARY.mat')
# exptSummary = exptSummary['exptSummary']
params = exptSummary['exptSummary']['params']
baselineWindow = np.ceil(params['baselineWindow_Glu_s']/params['frametime'])

# with h5py.File('/root/capsule/scratch/Matlab/SIMULATION_scan_00003_Standard2_SIMPARAMS.mat', 'r') as f:
#     params = f['param'][()]



IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [34]:
exptSummary['exptSummary'].dtype

dtype([('meanIM', 'O'), ('actIM', 'O'), ('params', 'O'), ('dFerr', 'O'), ('matchFilt', 'O'), ('events', 'O'), ('dF', 'O'), ('dF2', 'O'), ('dFls', 'O'), ('F0', 'O'), ('footprints', 'O'), ('dFF', 'O'), ('dFF2', 'O'), ('dFFls', 'O'), ('fns', 'O'), ('dr', 'O'), ('peaks', 'O'), ('perTrialMeanIMs', 'O'), ('perTrialActIms', 'O'), ('perTrialAlignmentOffsets', 'O')])

In [66]:
params[0][0][0][0]['baselineWindow_Glu_s'][0]

array([2], dtype=uint8)

In [68]:
data = np.array(params[0][0][0], dtype=[(name, 'O') for name in params[0][0][0][0].dtype.names])
